In [48]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.integrate import solve_ivp
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm import tqdm

In [2]:
# Double Pendulum Dynamics
def double_pendulum(t, y, l1, l2, m1, m2, g):
    theta1, z1, theta2, z2 = y
    delta = theta2 - theta1
    denom1 = (m1 + m2) * l1 - m2 * l1 * np.cos(delta) ** 2
    denom2 = (l2 / l1) * denom1

    dydt = np.zeros_like(y)
    dydt[0] = z1
    dydt[1] = (
        (m2 * l1 * z1 ** 2 * np.sin(delta) * np.cos(delta)
         + m2 * g * np.sin(theta2) * np.cos(delta)
         + m2 * l2 * z2 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta1))
        / denom1
    )
    dydt[2] = z2
    dydt[3] = (
        (-m2 * l2 * z2 ** 2 * np.sin(delta) * np.cos(delta)
         + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
         - (m1 + m2) * l1 * z1 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta2))
        / denom2
    )
    return dydt

In [5]:
# Main Workflow
n_pendulums = 100
output_dir = "video_dataset"

In [6]:
# Generate Dataset
def generate_dataset(n_pendulums, dt, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    dataset = []

    for i in tqdm(range(n_pendulums), desc="Generating dataset"):
        l1, l2 = np.random.uniform(0.5, 2.0, 2)
        m1, m2 = np.random.uniform(0.5, 2.0, 2)
        g = 9.81
        y0 = np.random.uniform(-np.pi, np.pi, 4)
        t_span = (0, 10)
        t_eval = np.linspace(t_span[0], t_span[1], int(10 / dt))

        sol = solve_ivp(double_pendulum, t_span, y0, t_eval=t_eval, args=(l1, l2, m1, m2, g), method='RK45')
        data = np.column_stack((sol.y[0], sol.y[2]))

        gif_path = os.path.join(output_dir, f"pendulum_{i}.gif")
        generate_gif(data, l1, l2, gif_path)

        dataset.append((data, l1, l2, m1, m2, g, gif_path))

    return dataset

In [8]:
# Generate GIFs
def generate_gif(data, l1, l2, save_path):
    theta1, theta2 = data[:, 0], data[:, 1]
    x1 = l1 * np.sin(theta1)
    y1 = -l1 * np.cos(theta1)
    x2 = x1 + l2 * np.sin(theta2)
    y2 = y1 - l2 * np.cos(theta2)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    line, = ax.plot([], [], 'o-', lw=2)

    def update(frame):
        line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
        return line,

    ani = FuncAnimation(fig, update, frames=len(theta1), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [9]:
dataset = generate_dataset(n_pendulums, dt=0.01, output_dir=output_dir)

Generating dataset: 100%|██████████| 100/100 [58:09<00:00, 34.90s/it]


In [10]:
# Dataset Preparation
class DoublePendulumDataset(Dataset):
    def __init__(self, dataset, seq_len, pred_len):
        self.dataset = dataset
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data, l1, l2, m1, m2, g, gif_path = self.dataset[idx]

        gif = Image.open(gif_path)
        frames = []
        try:
            while True:
                frame = gif.convert("RGB")
                frames.append(np.array(frame))
                gif.seek(gif.tell() + 1)
        except EOFError:
            pass

        positions = np.column_stack([data[:, 0], data[:, 1]])

        input_seq = positions[:self.seq_len]
        target_seq = positions[self.seq_len:self.seq_len + self.pred_len]

        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32), gif_path

In [11]:
seq_len = 90
pred_len = 10
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_dataset = DoublePendulumDataset(train_data, seq_len, pred_len)
val_dataset = DoublePendulumDataset(val_data, seq_len, pred_len)
test_dataset = DoublePendulumDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [12]:
input_dim = 2
hidden_dim = 64
output_dim = 2
num_layers = 2

In [13]:
# Model Definition
class LSTMPendulumPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMPendulumPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out.unsqueeze(1)

In [14]:
# Training the Model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, targets, _ in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")

In [72]:
# Generate Prediction GIFs
def generate_prediction_gif(model, test_loader, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    true_motion_dir = os.path.join(output_dir, "true_motion")
    pred_motion_dir = os.path.join(output_dir, "predicted_motion")
    os.makedirs(true_motion_dir, exist_ok=True)
    os.makedirs(pred_motion_dir, exist_ok=True)

    model.eval()

    def compute_positions(theta):
        x1 = np.sin(theta[0])
        y1 = -np.cos(theta[0])
        x2 = x1 + np.sin(theta[1])
        y2 = y1 - np.cos(theta[1])
        return np.array([x1, y1, x2, y2])

    for i, (inputs, targets, _) in enumerate(test_loader):
        with torch.no_grad():
            # Generate sequential predictions
            inputs_seq = inputs.squeeze(0).numpy()
            predictions = []
            current_input = inputs_seq[0]  # Initial state

            # Debug: Initial input shape
            print(f"Sample {i}: Initial input shape: {current_input.shape}")

            for _ in range(targets.shape[1]):  # Number of time steps
                # Reshape current_input to match model input shape
                model_input = torch.tensor(current_input).unsqueeze(0).unsqueeze(1)  # Shape (1, 1, input_size)
                current_pred = model(model_input).squeeze(0).numpy()  # Shape (output_size,)
                
                # Debug: Log shapes before concatenation
                print(f"Current input shape: {current_input.shape}, Current prediction shape: {current_pred.shape}")

                current_pred = current_pred.flatten()  # Ensure 1D shape for concatenation
                current_input = np.concatenate([current_input[2:], current_pred])  # Shift and append
                predictions.append(current_pred)

            predictions = np.array(predictions).T  # Shape (2, N)

        # Reshape true positions
        true_positions = targets.squeeze(0).numpy().reshape(2, -1)

        # Debugging: Print shapes
        print(f"Sample {i}: Reshaped True positions shape {true_positions.shape}, Reshaped Predicted positions shape {predictions.shape}")

        # Compute positional data
        true_pos = compute_positions(true_positions)
        pred_pos = compute_positions(predictions)

        num_frames = min(true_pos.shape[1], pred_pos.shape[1])

        def create_gif(data, save_path, label, style):
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.set_xlim(-2.5, 2.5)
            ax.set_ylim(-2.5, 2.5)
            ax.set_aspect('equal')
            line, = ax.plot([], [], style, lw=2, label=label)
            ax.legend()

            def update(frame):
                x = [0, data[0, frame], data[2, frame]]
                y = [0, data[1, frame], data[3, frame]]
                line.set_data(x, y)
                return line,

            ani = FuncAnimation(fig, update, frames=num_frames, blit=True, interval=50)
            ani.save(save_path, fps=20, writer='pillow')
            plt.close(fig)

        # Create True Motion GIF
        true_gif_path = os.path.join(true_motion_dir, f"true_motion_{i}.gif")
        create_gif(true_pos, true_gif_path, "True Motion", "o-")

        # Create Predicted Motion GIF
        pred_gif_path = os.path.join(pred_motion_dir, f"predicted_motion_{i}.gif")
        create_gif(pred_pos, pred_gif_path, "Predicted Motion", "o--")



In [16]:
model = LSTMPendulumPredictor(input_dim, hidden_dim, output_dim, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1/20:   0%|          | 0/3 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32, 10, 2])) that is different to the input size (torch.Size([32, 1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/20:  67%|██████▋   | 2/3 [03:00<01:29, 89.91s/it]C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([16, 10, 2])) that is different to the input size (torch.Size([16, 1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self

Epoch 1, Train Loss: 5.3736090660095215, Val Loss: 4.52817964553833


Epoch 2/20: 100%|██████████| 3/3 [03:45<00:00, 75.24s/it]


Epoch 2, Train Loss: 5.318526983261108, Val Loss: 4.28372049331665


Epoch 3/20: 100%|██████████| 3/3 [03:45<00:00, 75.22s/it]


...

Epoch 18, Train Loss: 1.3085393210252125, Val Loss: 0.3936749994754791


Epoch 19/20: 100%|██████████| 3/3 [03:55<00:00, 78.67s/it]


Epoch 19, Train Loss: 1.3139463663101196, Val Loss: 0.3121100664138794


Epoch 20/20: 100%|██████████| 3/3 [03:49<00:00, 76.55s/it]


Epoch 20, Train Loss: 1.5473692814509075, Val Loss: 0.27944207191467285


In [73]:
generate_prediction_gif(model, test_loader, output_dir="predicted_gifs")

Sample 0: Initial input shape: (2,)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Sample 0: Reshaped True positions shape (2, 10), Reshaped Predicted positions shape (2, 10)
Sample 1: Initial input shape: (2,)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1, 2)
Current input shape: (2,), Current prediction shape: (1,